In [2]:
!{sys.executable} -m pip install dask
!{sys.executable} -m pip install --upgrade google-cloud-storage
!{sys.executable} -m pip install praw
!{sys.executable} -m pip install ipyparallel

import requests
import time
import sys
import pandas as pd
import os
import praw

Requirement already up-to-date: google-cloud-storage in /Users/willyraedy/anaconda3/lib/python3.7/site-packages (1.23.0)


In [3]:
df = pd.read_csv('AllReddits_2019-10-18.csv', encoding='latin1')

In [7]:
df.describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.72, 0.74, 0.76, 0.78, 0.8, 0.82, 0.84, 0.86, 0.88, 0.9, 0.92, 0.94, 0.96, 0.98])

,subs
count,1.686748e+06
mean,1.412621e+03
std,1.049948e+05
min,-1.000000e+00
10%,1.000000e+00
20%,1.000000e+00
30%,1.000000e+00
40%,1.000000e+00
50%,2.000000e+00
60%,4.000000e+00


In [54]:
df['in_latino_network'] = None
df['description_html'] = None

In [55]:
df.shape

(1686748, 6)

In [96]:
def fetch_description_for_subset(subset_index):
    import time
    import pandas as pd
    import sys
    import praw
    import glob
    import math

    temp = []

    for filename in glob.glob('./results/FirstSuccessfulScrape/*.csv'):
        df = pd.read_csv(filename)
        temp.append(df)

    frame = pd.concat(temp)
    df = frame[(frame['subs'] > 10) & (pd.isnull(frame['description_html']))]
    
    bucket_size = math.ceil(df.shape[0] / 10)

    def make_fetch_description(subset):
        credentials = [('f_wJd_CV9WBtxg', '23iMmeQKcovVMizPwx_5Nn4cg5M'), ('0k-ZbZ3PB3OOFg', 'M8_N4FiuaectjNnyNnarslG4Tjc'), ('oeXaoZM0yqyNIg', 'lsaJWiCnu_jIvZO_Y8BmL6wmmFs'), ('IhOxGlRhSSrwxA', 'VqV6DdUclLK4czmYgHw9C6IJUME'), ('SeygGduGXMkELA', 'zWz862BaRe7viZBXNdQZobOw9mo'), ('mHTRKgTy1BIoXg', 'YmIUEfGQw70WIqyQMGTEhcwpazI'), ('wlH5TZ37dlivRg', 'vKT6nTWDU47eNcB4x74HuTUPujk'), ('n1ipbojZDxQ2hw', 'R_sx68Ac40vs2fpyiox90aLQDjU'), ('RC5iOV7vvMM50g', '6OVgP2BQ1PzWRSGB5SJWKQf1AWQ'), ('p14y-jjK1_x_pA', 'sadN8Sfq0kAOF2uFyxQylRXqPmo')]
        credential = credentials[subset]
        client_id = credential[0]
        client_secret = credential[1]

        reddit = praw.Reddit(client_id=client_id, client_secret=client_secret,
                    password='n4JDYt34xJakggnMQD2d7kxw', user_agent=f'script:melis-thesis:v0.0.1-agent-{subset} (by /u/wilburRay)',
                    username='wilburRay')

        def fetch_description_html(reddit_name):
            print(f'starting {reddit_name}')
            try:
                sub = reddit.subreddit(reddit_name)
                print(f'finishing {reddit_name}')
                return sub.description_html
            except KeyboardInterrupt:
                raise
            except:
                e = sys.exc_info()[0]
                print(e)

        return fetch_description_html
    
    starting_index = subset_index * bucket_size
    ending_index = (subset_index + 1) * bucket_size - 1
    
    fetch_description = make_fetch_description(subset_index)
    
    subset = df.loc[starting_index:ending_index]
    subset['description_html'] = subset['real_name'].map(fetch_description)
    
    subset.to_csv(f'/Users/willyraedy/Sync/SideProjects/RedditResearch/results/results-{subset_index}-{time.time()}.csv')
    
    return subset

In [57]:
from ipyparallel import Client
rc = Client()
dv = rc[:]
dv.map_sync(fetch_description_for_subset, range(10))

[                   real_name  \
 0                       pics   
 1                      funny   
 2                   politics   
 3                     gaming   
 4                  AskReddit   
 5                  worldnews   
 6                     videos   
 7                       IAmA   
 8              todayilearned   
 9                        WTF   
 10                       aww   
 11                   atheism   
 12                technology   
 13             AdviceAnimals   
 14                   science   
 15                     Music   
 16                    movies   
 17                    bestof   
 18                  gonewild   
 19                     trees   
 20       fffffffuuuuuuuuuuuu   
 21                      gifs   
 22           leagueoflegends   
 23                     4chan   
 24                   pokemon   
 25            thewalkingdead   
 26              reactiongifs   
 27                 starcraft   
 28                      nsfw   
 29      I

In [65]:
df3 = pd.read_csv('./results/results-8-1573165587.677593.csv', encoding='latin1')

In [67]:
df3

,Unnamed: 0,real_name,desc,created_date,subs,description_html
0,1349400,StrangerThingsTheGame,NaN,2018-12-07,1,NaN
1,1349401,liveufc231hd,NaN,2018-12-07,1,NaN
2,1349402,epicgamestore,NaN,2018-12-07,2,NaN
3,1349403,waughgaming,the one true epic waugh gaming worm,2018-12-07,2,"<!-- SC_OFF --><div class=""md""><p>the one true..."
4,1349404,JapeEgg,Only the most compressed memes.,2018-12-07,1,"<!-- SC_OFF --><div class=""md""><p>Only the mos..."
5,1349405,Bugscraft,Reddit community for the mobile game Bugscraft,2018-12-07,2,"<!-- SC_OFF --><div class=""md""><p>Reddit commu..."
6,1349406,mapleservers,A subreddit for the discussion of MapleStory p...,2018-12-07,205,"<!-- SC_OFF --><div class=""md""><p><strong>A li..."
7,1349407,CalculatorMasteRace,This subreddit is only for true gamers. (the p...,2018-12-07,2,"<!-- SC_OFF --><div class=""md""><p>This subredd..."
8,1349408,notcapitalism,"Just, not capitalism",2018-12-07,1,"<!-- SC_OFF --><div class=""md""><p>Just, not ca..."
9,1349409,Farcry_NewDawn,Subreddit about all and anything New Dawn.,2018-12-07,1,"<!-- SC_OFF --><div class=""md""><p>A subreddit ..."


In [70]:
import glob

temp = []

for filename in glob.glob('./results/*.csv'):
    df = pd.read_csv(filename)
    temp.append(df)

frame = pd.concat(temp)

frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1686748 entries, 0 to 168674
Data columns (total 6 columns):
Unnamed: 0          1686748 non-null int64
real_name           1686746 non-null object
desc                1242787 non-null object
created_date        1686748 non-null object
subs                1686748 non-null int64
description_html    942445 non-null object
dtypes: int64(2), object(4)
memory usage: 90.1+ MB


In [74]:
def filter_network_reddits(dataframe):
    return dataframe[dataframe['description_html'].str.contains('latinoamerica', na=False)]

In [75]:
in_network = filter_network_reddits(frame)

In [ ]:
in_network.to_csv('network.csv')

In [8]:
frame.head(40)

NameError: name 'frame' is not defined

In [80]:
at_least_10 = frame[frame['subs'] > 10]

In [81]:
at_least_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429487 entries, 11 to 168667
Data columns (total 6 columns):
Unnamed: 0          429487 non-null int64
real_name           429486 non-null object
desc                344079 non-null object
created_date        429487 non-null object
subs                429487 non-null int64
description_html    295288 non-null object
dtypes: int64(2), object(4)
memory usage: 22.9+ MB


In [94]:
at_least_10_and_missing = frame[(frame['subs'] > 10) & (pd.isnull(frame['description_html']))]

In [95]:
at_least_10_and_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134199 entries, 15 to 168653
Data columns (total 6 columns):
Unnamed: 0          134199 non-null int64
real_name           134198 non-null object
desc                93010 non-null object
created_date        134199 non-null object
subs                134199 non-null int64
description_html    0 non-null object
dtypes: int64(2), object(4)
memory usage: 7.2+ MB


In [ ]:
dv.map_sync(fetch_description_for_subset, range(10))